In [1]:
import pandas as pd
import warnings
import json
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
os.chdir('../')
from model import *
os.chdir('../../data/')

In [2]:
transactions = pd.read_pickle('../data/compressed_data/transactions_train.pkl')
customers = pd.read_pickle('../data/compressed_data/customers.pkl')
articles = pd.read_pickle('../data/compressed_data/articles.pkl')

transactions['week'] = 104 - (transactions.t_dat.max() - transactions.t_dat).dt.days // 7

print('First week num: ', transactions.week.min(), '\nLast week num: ', transactions.week.max(), '\n')

First week num:  0 
Last week num:  104 



In [3]:
# Test week is week after last week in train data
test_week = transactions.week.max() + 1

# Filter transactions to last 10 weeks (most recent data)
transactions = transactions[transactions.week > transactions.week.max() - 10]

In [4]:
transactions.columns

Index(['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id',
       'week'],
      dtype='object')

In [5]:
# Load radek's candidates
candidates_last_purchase = pd.read_csv('../data/candidates/radek_last_purchase.csv')
candidates_bestsellers = pd.read_csv('../data/candidates/radek_bestsellers.csv')
bestsellers_previous_week = pd.read_csv('../data/candidates/radek_bestsellers_previous_week.csv')

# Load my candidates

In [6]:
# Load my candidates
candidates_niw_loaded = pd.read_csv('../data/candidates_200_ranks/niw_candidates_garment_group_name.csv')

## Submission Loop -- check best k for kaggle

In [7]:
columns_to_use = [
    'article_id', 
    'product_type_no', 
    'graphical_appearance_no', 
    'colour_group_code', 
    'perceived_colour_value_id',
    'perceived_colour_master_id', 
    'department_no', 
    'index_code',
    'index_group_no', 
    'section_no', 
    'garment_group_no', 
    'FN', 
    'Active',
    'club_member_status', 
    'fashion_news_frequency', 
    'age', 
    'postal_code', 
    'bestseller_rank',
    'not_interacted_weekly_rank'
]

model_params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'boosting_type': 'dart',
    'n_estimators': 1,
    'importance_type': 'gain'
}

In [9]:
# Get bestsellers from previous week
bestsellers_last_week = \
    bestsellers_previous_week[bestsellers_previous_week['week'] == bestsellers_previous_week['week'].max()]['article_id'].tolist()

niw_ranks = candidates_niw_loaded[['week', 'article_id', 'not_interacted_weekly_rank']].drop_duplicates()

for k in range(20, 201, 10):
    print('k = ', k)

    # Get top k similar not bought articles for each customer
    candidates_niw = candidates_niw_loaded.groupby(['week', 'customer_id']).head(k)\
        .drop(columns=['strategy', 'not_interacted_weekly_rank'])

    candidates_niw['week'] = candidates_niw['week'] + 1

    candidates_niw['t_dat'] = '2020-07-15'
    candidates_niw['price'] = 0
    candidates_niw['sales_channel_id'] = 2

    # Prepare data for model
    train_X, train_y, test_X, test, train_baskets = prepare_data(
        transactions,
        bestsellers_previous_week,
        candidates=[candidates_last_purchase, candidates_bestsellers, candidates_niw], 
        features=[customers, articles, niw_ranks], 
        cols_to_use=columns_to_use
        )
    
    del candidates_niw
    gc.collect()
    
    # Train model
    ranker = train_model(
        train_X, 
        train_y, 
        train_baskets, 
        model_params, 
        columns_to_use, 
        show_importance=10
    )

    gc.collect()

    # Make submission
    make_submission(customers, test, test_X, ranker, bestsellers_last_week, f'submission_niw_garment_group_{k}')

    del train_X, train_y, test_X, test, train_baskets, ranker
    gc.collect()



k =  20
Percentage of real transactions:  0.07313803995527318
Mergining features...
Done.
Sorting data...
Done.
Preparing for training...
Done.
Training model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.238352 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1133
[LightGBM] [Info] Number of data points in the train set: 24531862, number of used features: 19
Feature importance:
bestseller_rank 0.8371640800628387
not_interacted_weekly_rank 0.10473969797965298
index_code 0.016873756326542606
department_no 0.010063857078205387
garment_group_no 0.007863416770449822
article_id 0.007464027003561572
product_type_no 0.005771634475179649
perceived_colour_value_id 0.0029643150920194146
colour_group_code 0.0026579830286630347
perceived_colour_master_id 0.0023870990772154634
Starting submission process...
Calculating predictions...
Done.
Creat

100%|██████████| 58.0M/58.0M [00:27<00:00, 2.25MB/s]


Successfully submitted to H&M Personalized Fashion RecommendationsSubmission saved and submitted to Kaggle.
k =  30
Percentage of real transactions:  0.05909607318861378
Mergining features...
Done.
Sorting data...
Done.
Preparing for training...
Done.
Training model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.292496 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1123
[LightGBM] [Info] Number of data points in the train set: 31262091, number of used features: 19
Feature importance:
bestseller_rank 0.8033436380115033
not_interacted_weekly_rank 0.12861269065385017
index_code 0.02472004202813619
department_no 0.014068760379298304
article_id 0.010128278379545993
garment_group_no 0.008591878745881545
product_type_no 0.0036044641598853033
graphical_appearance_no 0.002423293233135136
colour_group_code 0.0016568054770928996
section_no 0.

100%|██████████| 58.1M/58.1M [00:26<00:00, 2.29MB/s]


Successfully submitted to H&M Personalized Fashion RecommendationsSubmission saved and submitted to Kaggle.
k =  40
Percentage of real transactions:  0.04950112336871991
Mergining features...
Done.
Sorting data...
Done.
Preparing for training...
Done.
Training model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.352327 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1110
[LightGBM] [Info] Number of data points in the train set: 38011697, number of used features: 19
Feature importance:
bestseller_rank 0.7709836757172868
not_interacted_weekly_rank 0.15581477848393835
garment_group_no 0.022933084295029077
index_code 0.021607073279271728
department_no 0.013873314977602814
article_id 0.009380022795310717
product_type_no 0.0030197336094880335
graphical_appearance_no 0.0014912539747487213
perceived_colour_master_id 0.0008970628673237806
Ac

...

Successfully submitted to H&M Personalized Fashion RecommendationsSubmission saved and submitted to Kaggle.
k =  190
Percentage of real transactions:  0.014243014138486366
Mergining features...
Done.
Sorting data...
Done.
Preparing for training...
Done.
Training model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 2.032448 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1047
[LightGBM] [Info] Number of data points in the train set: 140630610, number of used features: 19
Feature importance:
bestseller_rank 0.6137178426727896
index_code 0.20787980773433887
not_interacted_weekly_rank 0.1025118978948199
garment_group_no 0.0553542253948239
department_no 0.0123705481630428
article_id 0.0031468693148376395
section_no 0.003001512698268842
colour_group_code 0.0013389823928122132
graphical_appearance_no 0.0006783137342661638
Active 0.0
Starting

100%|██████████| 57.8M/57.8M [00:28<00:00, 2.11MB/s]  


Successfully submitted to H&M Personalized Fashion RecommendationsSubmission saved and submitted to Kaggle.
k =  200
Percentage of real transactions:  0.013594696283440014
Mergining features...
Done.
Sorting data...
Done.
Preparing for training...
Done.
Training model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.993001 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1043
[LightGBM] [Info] Number of data points in the train set: 147497523, number of used features: 19
Feature importance:
bestseller_rank 0.6070292238819979
index_code 0.21971102000857276
not_interacted_weekly_rank 0.0982471308561217
garment_group_no 0.05433521271418926
department_no 0.01271535903341661
section_no 0.003040677719482183
article_id 0.0030188398992914314
colour_group_code 0.001234212062094655
graphical_appearance_no 0.0006683238248335508
Active 0.0
Startin

100%|██████████| 57.8M/57.8M [00:26<00:00, 2.29MB/s]


Successfully submitted to H&M Personalized Fashion RecommendationsSubmission saved and submitted to Kaggle.


In [10]:
!kaggle competitions submissions -c h-and-m-personalized-fashion-recommendations | head -n 21

fileName                                 date                 description                       status    publicScore  privateScore  
---------------------------------------  -------------------  --------------------------------  --------  -----------  ------------  
submission_niw_garment_group_200.csv.gz  2023-12-14 17:15:57  submission_niw_garment_group_200  complete  0.01633      0.01551       
submission_niw_garment_group_190.csv.gz  2023-12-14 17:03:59  submission_niw_garment_group_190  complete  0.01633      0.01551       
submission_niw_garment_group_180.csv.gz  2023-12-14 16:52:11  submission_niw_garment_group_180  complete  0.01633      0.01551       
submission_niw_garment_group_170.csv.gz  2023-12-14 16:40:45  submission_niw_garment_group_170  complete  0.01072      0.0107        
submission_niw_garment_group_160.csv.gz  2023-12-14 16:30:52  submission_niw_garment_group_160  complete  0.01068      0.01069       
submission_niw_garment_group_150.csv.gz  2023-12-14 16:19:14  